In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
pip install torchvision

In [ ]:
conda install torchvision -c pytorch

In [ ]:
import torchvision
print(torchvision.__version__)

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define the paths to your directories
train_dir = r'\Users\DELL\Downloads\archive\chest_xray\chest_xray\train'
val_dir = r'\Users\DELL\Downloads\archive\chest_xray\chest_xray\val'
test_dir = r'\Users\DELL\Downloads\archive\chest_xray\chest_xray\test'

# Now you can list the classes
train_classes = os.listdir(train_dir)
val_classes = os.listdir(val_dir)
test_classes = os.listdir(test_dir)

In [ ]:
!pip install scikit-learn

In [ ]:
import shutil
import random

In [ ]:
pip install torch

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

In [ ]:
!pip install scikit-learn

In [ ]:
import torch
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
train_dir = '/kaggle/input/chest-xray-pneumonia/'
val_dir = '/kaggle/input/chest-xray-pneumonia/'
test_dir = '/kaggle/input/chest-xray-pneumonia/'

In [ ]:
chest_xray_dir = os.path.join(train_dir, 'chest_xray')

In [ ]:
train_dir = os.path.join(chest_xray_dir, 'train')
val_dir = os.path.join(chest_xray_dir, 'val')
test_dir = os.path.join(chest_xray_dir, 'test')

In [ ]:
def plot_class_distribution(data, title):
    classes = list(data.keys())
    counts = list(data.values())
    
    plt.figure(figsize=(6, 4))
    plt.bar(classes, counts, color=['blue', 'orange'])
    plt.title(title)
    plt.xlabel('Classes')
    plt.ylabel('Número de Imagens')
    plt.show()

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cpu'):
    history = {'train_loss': [], 'val_loss': [],
               'train_accuracy': [], 'val_accuracy': [],
               'train_f1': [], 'val_f1': [],
               'train_precision': [], 'val_precision': [],
               'train_recall': [], 'val_recall': []}
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 20)
        
        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
        train_loss = running_loss / len(train_loader)
        train_f1 = f1_score(all_labels, all_preds, average='weighted')
        train_precision = precision_score(all_labels, all_preds, average='weighted')
        train_recall = recall_score(all_labels, all_preds, average='weighted')
        train_accuracy = (torch.tensor(all_preds) == torch.tensor(all_labels)).sum().item() / len(all_labels)
        
        history['train_loss'].append(train_loss)
        history['train_accuracy'].append(train_accuracy)
        history['train_f1'].append(train_f1)
        history['train_precision'].append(train_precision)
        history['train_recall'].append(train_recall)

        model.eval()
        val_loss = 0.0
        all_val_preds = []
        all_val_labels = []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, preds = torch.max(outputs, 1)
                all_val_preds.extend(preds.cpu().numpy())
                all_val_labels.extend(labels.cpu().numpy())
        
        val_loss = val_loss / len(val_loader)
        val_f1 = f1_score(all_val_labels, all_val_preds, average='weighted')
        val_precision = precision_score(all_val_labels, all_val_preds, average='weighted')
        val_recall = recall_score(all_val_labels, all_val_preds, average='weighted')
        val_accuracy = (torch.tensor(all_val_preds) == torch.tensor(all_val_labels)).sum().item() / len(all_val_labels)
        
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)
        history['val_f1'].append(val_f1)
        history['val_precision'].append(val_precision)
        history['val_recall'].append(val_recall)

        print(f'Treino: Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, F1-score: {train_f1:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}')
        print(f'Validação: Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, F1-score: {val_f1:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}')
        print()
    
    return history

In [ ]:
def evaluate_model_on_test(model, test_loader, criterion, device='cpu'):
    model.eval()  
    
    test_loss = 0.0
    all_test_preds = []
    all_test_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            _, preds = torch.max(outputs, 1)
            all_test_preds.extend(preds.cpu().numpy())
            all_test_labels.extend(labels.cpu().numpy())

    test_loss = test_loss / len(test_loader)
    test_f1 = f1_score(all_test_labels, all_test_preds, average='weighted')
    test_precision = precision_score(all_test_labels, all_test_preds, average='weighted')
    test_recall = recall_score(all_test_labels, all_test_preds, average='weighted')
    test_accuracy = (torch.tensor(all_test_preds) == torch.tensor(all_test_labels)).sum().item() / len(all_test_labels)

    print(f'Teste: Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, F1-score: {test_f1:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}')
    
    return test_loss, test_accuracy, test_f1, test_precision, test_recall


In [ ]:
def plot_metrics(history):
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss
    plt.figure(figsize=(12, 6))
    plt.subplot(2, 3, 1)
    plt.plot(epochs, history['train_loss'], label='Treino Loss')
    plt.plot(epochs, history['val_loss'], label='Validação Loss')
    plt.title('Loss')
    plt.legend()

    # Accuracy
    plt.subplot(2, 3, 2)
    plt.plot(epochs, history['train_accuracy'], label='Treino Accuracy')
    plt.plot(epochs, history['val_accuracy'], label='Validação Accuracy')
    plt.title('Acurácia')
    plt.legend()

    # F1-Score
    plt.subplot(2, 3, 3)
    plt.plot(epochs, history['train_f1'], label='Treino F1-Score')
    plt.plot(epochs, history['val_f1'], label='Validação F1-Score')
    plt.title('F1-Score')
    plt.legend()

    # Precision
    plt.subplot(2, 3, 4)
    plt.plot(epochs, history['train_precision'], label='Treino Precision')
    plt.plot(epochs, history['val_precision'], label='Validação Precision')
    plt.title('Precision')
    plt.legend()

    # Recall
    plt.subplot(2, 3, 5)
    plt.plot(epochs, history['train_recall'], label='Treino Recall')
    plt.plot(epochs, history['val_recall'], label='Validação Recall')
    plt.title('Recall')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def evaluate_and_plot_confusion_matrix(model, test_loader, device='cpu'):
    model.eval()  
    all_test_preds = []
    all_test_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            
            _, preds = torch.max(outputs, 1)
            all_test_preds.extend(preds.cpu().numpy())
            all_test_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_test_labels, all_test_preds)
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
    plt.title('Confusion Matrix - Test Set')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
def plot_confusion_matrix_validation(model, val_loader, device='cpu'):
    model.eval()  
    all_val_preds = []
    all_val_labels = []
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            
            _, preds = torch.max(outputs, 1)
            all_val_preds.extend(preds.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_val_labels, all_val_preds)
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
    plt.title('Confusion Matrix - Validation Set')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
def show_images(dataset_dir, class_name, num_images=5):
    class_path = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_path)[:num_images]
    
    plt.figure(figsize=(12, 6))
    for i, img_name in enumerate(images):
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path)
        plt.subplot(1, num_images, i + 1)
        plt.imshow(img, cmap='gray')
        plt.title(class_name)
        plt.axis('off')
    plt.show()


In [ ]:
def plot_histogram_of_intensities(dataset_dir, class_name, num_images=5):
    class_path = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_path)[:num_images]
    
    plt.figure(figsize=(10, 5))
    for i, img_name in enumerate(images):
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path).convert('L')  
        img_np = np.array(img).flatten()  
        
        plt.subplot(1, num_images, i + 1)
        plt.hist(img_np, bins=50, color='blue', alpha=0.7)
        plt.title(f'Histograma - {class_name} {i+1}')
        plt.xlabel('Valor de Pixel')
        plt.ylabel('Frequência')
    plt.tight_layout()
    plt.show()

In [ ]:

train_classes = os.listdir(train_dir)
val_classes = os.listdir(val_dir)
test_classes = os.listdir(test_dir)

In [ ]:
print(f'Classes in training: {train_classes}')
print(f'Classes em validation: {val_classes}')
print(f'Classes em test: {test_classes}')


In [ ]:
# Filter out non-directory entries
train_classes = [cls for cls in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, cls))]
val_classes = [cls for cls in os.listdir(val_dir) if os.path.isdir(os.path.join(val_dir, cls))]
test_classes = [cls for cls in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, cls))]

# Count the number of files in each class directory
train_count = {cls: len(os.listdir(os.path.join(train_dir, cls))) for cls in train_classes}
val_count = {cls: len(os.listdir(os.path.join(val_dir, cls))) for cls in val_classes}
test_count = {cls: len(os.listdir(os.path.join(test_dir, cls))) for cls in test_classes}


In [ ]:
print(f'Training: {train_count}')
print(f'Validation: {val_count}')
print(f'Test: {test_count}')


In [ ]:
train_count = {'NORMAL': 1341, 'PNEUMONIA': 3875}  # Exemplo de contagem real
val_count = {'NORMAL': 8, 'PNEUMONIA': 8}  # Exemplo de contagem real
test_count = {'NORMAL': 234, 'PNEUMONIA': 390}  # Exemplo de contagem real


In [ ]:
# Define the function to plot class distribution
def plot_class_distribution(class_count, title):
    classes = list(class_count.keys())
    counts = list(class_count.values())

    plt.figure(figsize=(10, 5))
    plt.bar(classes, counts, color='skyblue')
    plt.title(title)
    plt.xlabel('Classes')
    plt.ylabel('Number of Samples')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_class_distribution(train_count, 'Classes Distribution - Training')
plot_class_distribution(val_count, 'Classes Distribution - Validation')
plot_class_distribution(test_count, 'Classes Distribution - Test')


In [ ]:
def show_images(directory, class_name, num_images=5):
    class_dir = os.path.join(directory, class_name)
    images = [img for img in os.listdir(class_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))][:num_images]  # Filter valid image files

    plt.figure(figsize=(15, 5))
    for i, image_name in enumerate(images):
        img_path = os.path.join(class_dir, image_name)
        img = mpimg.imread(img_path)
        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.axis('off')  # Hide axes
        plt.title(image_name)  # Optionally show image name
    plt.show()

In [ ]:
show_images(train_dir, 'NORMAL', num_images=5)
show_images(train_dir, 'PNEUMONIA', num_images=5)

In [ ]:
def plot_histogram_of_intensities(directory, class_name, num_images=3):
    class_dir = os.path.join(directory, class_name)
    images = [img for img in os.listdir(class_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))][:num_images]  # Filter valid image files

    plt.figure(figsize=(15, 5))
    for i, image_name in enumerate(images):
        img_path = os.path.join(class_dir, image_name)
        img = mpimg.imread(img_path)

        # Flatten the image array and plot histogram
        plt.subplot(1, num_images, i + 1)
        plt.hist(img.ravel(), bins=256, color='gray', alpha=0.7)
        plt.title(f'Histogram for {image_name}')
        plt.xlim([0, 255])  # Limiting x-axis to valid pixel intensity range
        plt.xlabel('Pixel Intensity')
        plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_histogram_of_intensities(train_dir, 'NORMAL', num_images=3)
plot_histogram_of_intensities(train_dir, 'PNEUMONIA', num_images=3)

In [ ]:
from torchvision import transforms

# Define the data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

In [ ]:
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms)

In [ ]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size


In [ ]:
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=True)

In [ ]:
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
print(f'Training Size: {len(train_subset)}')
print(f'Validation Size: {len(val_subset)}')
print(f'Test Size: {len(test_dataset)}')


In [ ]:
class CustomDenseNet(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomDenseNet, self).__init__()
        
        self.densenet = models.densenet121(pretrained=True)
        
        for param in self.densenet.parameters():
            param.requires_grad = False
        
        num_ftrs = self.densenet.classifier.in_features
        
        self.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),       
            nn.ReLU(),                       
            nn.Dropout(0.3),                 
            nn.Linear(512, 256),             
            nn.ReLU(),                       
            nn.BatchNorm1d(256),            
            nn.Linear(256, num_classes)      
        )
    
    def forward(self, x):
        x = self.densenet.features(x)
        x = nn.functional.relu(x, inplace=True)
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        
        x = self.classifier(x)
        return x

In [ ]:
num_classes = 2

In [ ]:
model = CustomDenseNet(num_classes)

In [ ]:
print(model)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

In [ ]:
num_epochs = 10

In [ ]:
history = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

In [ ]:
plot_metrics(history)

In [ ]:
plot_confusion_matrix_validation(model, val_loader, device)

In [ ]:
test_loss, test_accuracy, test_f1, test_precision, test_recall = evaluate_model_on_test(model, test_loader, criterion, device)


In [ ]:
evaluate_and_plot_confusion_matrix(model, test_loader, device)